### Deploiement de modèles Azure ML

definition **inférence**: l’inférence fait référence à l’utilisation d’un modèle entraîné pour prédire des étiquettes pour les nouvelles données sur lesquelles le modèle **n’a pas** été entraîné

### Déploiement d'un modèle en tant que service en temps réel

Azure Machine Learning:
- utilise des conteneurs comme mécanisme de déploiement, en empaquetant le modèle et le code pour l’utiliser en tant qu’image 
- qui peut être déployée sur un conteneur dans la cible de calcul que vous avez choisie

Etapes pour déployer un modèle en tant que service d’inférence en temps réel:
- 1) Inscrire un modèle entraîné
- 2) définir une configuration d'inférence 

##### Inscription du modèle 

Rappel : register_modelou Model.register

In [ ]:
from azureml.core import Model

classification_model = Model.register(workspace=ws,
                       model_name='classification_model',
                       model_path='model.pkl', # local path
                       description='A classification model')

run.register_model( model_name='classification_model',
                    model_path='outputs/model.pkl', # run outputs path
                    description='A classification model')

##### Définir une configuration d'inférence

Pour cela deux choses :
- 1) definir le script qui charge le modèle et l'applique aux nouvelles données
- 2) définir l'environnement

<font color='red'>1 - définition du script d'entrée ou script de scoring</font>

In [ ]:
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the registered model file and load it
    model_path = Model.get_model_path('classification_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Return the predictions as any JSON serializable format
    return predictions.tolist()


'''      Deux fonctions 

init()  : appelée lorsque le service est initialisé.
run(raw_data)  : appelée lorsque de nouvelles données sont envoyées au service.

'''



The web service will be hosted in a container,

<font color='red'>2 - définition de l'environnement</font>

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

# Add the dependencies for your model
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

# Save the environment config as a .yml file
env_file = 'service_files/env.yml'
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)


'''
- Utilisation de la classe CondaDependencies
- Ajouter tous les autres packages requis
- Sérialiser l’environnement en une chaîne 
- Enregistrer 
'''

<font color='red'>3 - Combinaison des deux en un **InferenceConfig**</font>

In [ ]:
from azureml.core.model import InferenceConfig

classifier_inference_config = InferenceConfig(runtime= "python",
                                              source_directory = 'service_files',
                                              entry_script="score.py",
                                              conda_file="env.yml")

##### Définir une configuration de déploiement 

But: consiste à configurer le calcul sur lequel le service sera déployé

In [ ]:
'''exemple :

Si on déploie sur un cluster AKS on doit pour celui-ci,
1- créer le cluster (étape 1)
2- créer une cible de calcul (étape 2)

'''

from azureml.core.compute import ComputeTarget, AksCompute

cluster_name = 'aks-cluster'
compute_config = AksCompute.provisioning_configuration(location='eastus') #étape 1
production_cluster = ComputeTarget.create(ws, cluster_name, compute_config) #étape 2
production_cluster.wait_for_completion(show_output=True)

In [ ]:
'''
Définition de leconfiguration de déploiement, qui définit la spécification de calcul spécifique à la 
cible pour le déploiement conteneurisé

'''

from azureml.core.webservice import AksWebservice

classifier_deploy_config = AksWebservice.deploy_configuration(cpu_cores = 1,
                                                              memory_gb = 1)

Remarque pour ACI: vous n’avez pas besoin de créer explicitement une cible de calcul ACI
                   vous devez utiliser la classe deploy_configuration de l’espace de nom      azureml.core.webservice.AciWebservice. 
                   vous pouvez utiliser l’espace de noms azureml.core.webservice.LocalWebservice pour configurer un service local basé sur Docker

##### Déploiement du modèle

In [ ]:
from azureml.core.model import Model

model = ws.models['classification_model']
service = Model.deploy(workspace=ws,
                       name = 'classifier-service',
                       models = [model],
                       inference_config = classifier_inference_config, #définit pendant la configuration d'inférence 
                       deployment_config = classifier_deploy_config, #définit dans la confi de déploiement
                       deployment_target = production_cluster) #définit dans la confi de déploiement ac config calcul
service.wait_for_deployment(show_output = True)

Remarque: Pour les services ACI ou locaux, vous pouvez omettre le paramètre deployment_target ou le définir sur None



ACI c'est plus en test ou en développement et AZure Kubernetes Service pour la production

### Utilisation d'un service d'inférence en teps réel

##### Utilisation du SDK Azure Machine Learning


Utilisation du SDK Azure Machine Learning pour appeler un service web

En règle générale, vous envoyez des données à la méthode run au format JSON

In [ ]:
import json

# An array of new data cases
x_new = [[0.1,2.3,4.1,2.0],
         [0.2,1.8,3.9,2.1]]

# Convert the array to a serializable list in a JSON document
json_data = json.dumps({"data": x_new})                 

# Call the web service, passing the input data
response = service.run(input_data = json_data)    #service étant le modèle déployé et transformé en service

# Get the predictions
predictions = json.loads(response)

# Print the predicted class for each case.
for i in range(len(x_new)):
    print (x_new[i]), predictions[i] )

##### Utilisation d'un REST endpoint

En production, la plupart des applications clientes n’incluront pas le SDK Azure Machine Learning et utiliseront le service via son interface REST

In [ ]:
# récupérer la propriété scoring_uri de l’objet Webservice dans le SDK

endpoint = service.scoring_uri
print(endpoint)


# Une fois le point de terminaison connu, vous pouvez utiliser une requête HTTP POST avec des données JSON 
# pour appeler le service


import requests
import json

# An array of new data cases
x_new = [[0.1,2.3,4.1,2.0],
         [0.2,1.8,3.9,2.1]]

# Convert the array to a serializable list in a JSON document
json_data = json.dumps({"data": x_new})

# Set the content type in the request headers
request_headers = { 'Content-Type':'application/json' }

# Call the service
response = requests.post(url = endpoint,
                         data = json_data,
                         headers = request_headers)

# Get the predictions from the JSON response
predictions = json.loads(response.json())

# Print the predicted class for each case.
for i in range(len(x_new)):
    print (x_new[i]), predictions[i] )

<font color='red'> Authentification </font>

Pour restreindre l’accès à vos services en appliquant une authentification deux types d’authentification :

- Clé : les demandes sont authentifiées en spécifiant la clé associée au service.
- Jeton : les demandes sont authentifiées en fournissant un jeton JWT (JSON Web Token).

Rmq: Par défaut, l’authentification est désactivée pour les services ACI et définie sur l’authentification basée sur les clés pour les services AKS

In [ ]:
'''En supposant que vous disposez d’une session authentifiée établie avec l’espace de travail, 
vous pouvez récupérer les clés d’un service avec la méthode get_keys de l’objet WebService associé au service'''

primary_key, secondary_key = service.get_keys()

'''
Pour l’authentification basée sur les jetons, votre application cliente doit utiliser l’authentification 
du principal du service pour vérifier son identité par le biais d’Azure AD (Azure Active Directory) et 
appeler la méthode get_token du service pour récupérer un jeton limité dans le temps.
'''



'''Puis appliquer lauthentification au rest endpoint'''

import requests
import json

# An array of new data cases
x_new = [[0.1,2.3,4.1,2.0],
         [0.2,1.8,3.9,2.1]]

# Convert the array to a serializable list in a JSON document
json_data = json.dumps({"data": x_new})

# Set the content type in the request headers
request_headers = { "Content-Type":"application/json",
                    "Authorization":"Bearer " + key_or_token }

# Call the service
response = requests.post(url = endpoint,
                         data = json_data,
                         headers = request_headers)

# Get the predictions from the JSON response
predictions = json.loads(response.json())

# Print the predicted class for each case.
for i in range(len(x_new)):
    print (x_new[i]), predictions[i] )

### Résolution des problèmes de déploiement de service

##### Vérifier l'état du service

In [ ]:
#par examination de son état 

from azureml.core.webservice import AksWebservice

# Get the deployed service
service = AciWebservice(name='classifier-service', workspace=ws)

# Check its state
print(service.state)   #Pour un service opérationnel, l’état doit être Sain.

##### Consulter les journaux de service

Si un service n’est pas sain ou si vous rencontrez des erreurs lors de son utilisation CONSULTER LES JOURNAUX:

In [ ]:
print(service.get_logs())

##### Déployer sur un conteneur local

Les erreurs de déploiement et d’exécution peuvent être plus faciles à diagnostiquer en déployant le service en tant que conteneur dans une instance Docker locale, pour pouvoir tester le service déployé localement à l’aide du SDK :

In [ ]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=8890)
service = Model.deploy(ws, 'test-svc', [model], inference_config, deployment_config)

In [ ]:
print(service.run(input_data = json_data))

résoudre les problèmes d’exécution:
- 1) en apportant des modifications au fichier de scoring référencé dans la configuration d’inférence
- 2) puis en rechargeant le service sans le redéployer

In [ ]:
service.reload()
print(service.run(input_data = json_data))

### Exercices

In [ ]:
# If you need to make a change and redeploy, you may need to delete unhealthy service using the following code:
#service.delete()

In [ ]:
import json

# This time our input is an array of two feature arrays
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes.
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

In [ ]:
#Let's determine the URL to which these applications must submit their requests:

endpoint = service.scoring_uri
print(endpoint)  #renvois un URL 